# 下載單一年的會考資料

In [13]:
import requests
import urllib.request
import os
from urllib.parse import urlparse


## 下載鑑別度、通過率和各科考題

In [25]:
urlList =  ['https://cap.rcpet.edu.tw/exam/110/110各科鑑別度.pdf',
             'https://cap.rcpet.edu.tw/exam/110/110各科通過率.pdf',
             'https://cap.rcpet.edu.tw/exam/110/110P_Chinese.pdf',
             'https://cap.rcpet.edu.tw/exam/110/110P_English.pdf',
             'https://cap.rcpet.edu.tw/exam/110/110P_Math.pdf',
             'https://cap.rcpet.edu.tw/exam/110/110P_Society.pdf',
             'https://cap.rcpet.edu.tw/exam/110/110P_Nature.pdf']
filename_list = []

for url in urlList:
    a = urlparse(url)
    print(a.path)
    
    pdf_path = url
    pdf_filename = os.path.basename(a.path)
    
    filename_list.append(pdf_filename)

    print(pdf_path)
    print(pdf_filename)

    chunk_size = 20000
    r = requests.get(pdf_path, stream=True)

    with open('會考題目/'+pdf_filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size):
            fd.write(chunk)

/exam/110/110各科鑑別度.pdf
https://cap.rcpet.edu.tw/exam/110/110各科鑑別度.pdf
110各科鑑別度.pdf
/exam/110/110各科通過率.pdf
https://cap.rcpet.edu.tw/exam/110/110各科通過率.pdf
110各科通過率.pdf
/exam/110/110P_Chinese.pdf
https://cap.rcpet.edu.tw/exam/110/110P_Chinese.pdf
110P_Chinese.pdf
/exam/110/110P_English.pdf
https://cap.rcpet.edu.tw/exam/110/110P_English.pdf
110P_English.pdf
/exam/110/110P_Math.pdf
https://cap.rcpet.edu.tw/exam/110/110P_Math.pdf
110P_Math.pdf
/exam/110/110P_Society.pdf
https://cap.rcpet.edu.tw/exam/110/110P_Society.pdf
110P_Society.pdf
/exam/110/110P_Nature.pdf
https://cap.rcpet.edu.tw/exam/110/110P_Nature.pdf
110P_Nature.pdf


## 解析鑑別度或通過率的PDF資料

In [1]:
import pdfplumber
import PyPDF2
import pandas as pd
from pandas import DataFrame


In [2]:
def extract_image(page):
    try:
        if '/XObject' in page['/Resources']:
            xObject = page['/Resources']['/XObject'].getObject()
            for obj in xObject:
                if xObject[obj]['/Subtype'] == '/Image':
                    size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                    data = xObject[obj].getData()
                    if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                        mode = "RGB"
                    else:
                        mode = "P"
                    # 保存图片的文件名前缀
                    img_pre = str(int(time.time()*1000))
                    if '/Filter' in xObject[obj]:
                        if xObject[obj]['/Filter'] == '/FlateDecode':
                            img = Image.frombytes(mode, size, data)
                            img.save(img_pre + ".png")
                        elif xObject[obj]['/Filter'] == '/DCTDecode':
                            img = open(img_pre + ".jpg", "wb")
                            img.write(data)
                            img.close()
                        elif xObject[obj]['/Filter'] == '/JPXDecode':
                            img = open(img_pre + ".jp2", "wb")
                            img.write(data)
                            img.close()
                        elif xObject[obj]['/Filter'] == '/CCITTFaxDecode':
                            img = open(img_pre + ".tiff", "wb")
                            img.write(data)
                            img.close()
                    else:
                        img = Image.frombytes(mode, size, data)
                        img.save(img_pre + ".png")
        else:
            print("no image")
    except:
        print("image extract fail")

In [3]:
def extract_content(pdf_path):
    # 内容提取，使用 pdfplumber 打开 PDF，用于提取文本
    with pdfplumber.open(pdf_path) as pdf_file:
        # 使用 PyPDF2 打开 PDF 用于提取图片
        pdf_image_reader = PyPDF2.PdfFileReader(open(pdf_path, "rb"))
        print(pdf_image_reader.getNumPages())

        content = ''
        # len(pdf.pages)为PDF文档页数，一页页解析
        for i in range(len(pdf_file.pages)):
            #print("第 %s 頁" % i)
            # pdf.pages[i] 是读取PDF文档第i+1页
            page_text = pdf_file.pages[i]
            # page.extract_text()函数即读取文本内容
            page_content = page_text.extract_text()
            if page_content:
                content = content + page_content + "\n"
                #print(page_content)

            # 提取图片
            #page_image = pdf_image_reader.getPage(pageNumber=i)
            #extract_image(page_image)
    return content

In [4]:
#將含有\n的text轉換成list  。第一維用'\n' 切割  第二維用 ''
def ConvertTextToList(string):
    out = []
    li = list(string.split('\n'))
    
    for l in li:
        m = list(l.split(' '))
        out.append(m)

    return out


In [5]:
# 將通過率、鑑別度pdf轉換成dataframe的函數

def convertPdfToDataframe(non_data_list,subject_questions_num_list,filename):
    text = extract_content(filename)
    #把text轉成list
    text_list = ConvertTextToList(text)

    columns_name = text_list[1]
    
    #檢查list的ele是否為想要的資料，用來寫出non_data_list
    #for i in range(len(text_list)):
        #print(i,'----',text_list[i])    
    
    #將text_list中非資料的部份刪掉，放到temp_text_list
    temp_text_list = []
    for i in range(len(text_list)):
        if i not in non_data_list:
            temp_text_list.append(text_list[i])

            


    #個別處理各科，把各科的通過率或鑑別度放回正確的位置
    for i in range(len(subject_questions_num_list)):
        for j in range(len(temp_text_list)):
            if j+1 > subject_questions_num_list[i]:
                temp_text_list[j].insert(i+1,'')

    
    #將 temp_text_list  轉為 df
    df = DataFrame (temp_text_list,columns=columns_name)
    return df

In [6]:
#將通過率和鑑別度的df轉型成['年度','類別','科目','題號','數值']
def convertDftoCleandataDf(df,year,statistical_type):
    statistical_list = []
    for idx, column in enumerate(df):
        #print(idx, column)
        if idx > 0:
            for idx_2, row in enumerate(df[column]):
                if len(row) > 0:
                    #print(year,statistical_type,column,idx_2+1,float(row))
                    statistical_list.append([ year,statistical_type,column,idx_2+1,float(row) ])

    new_df = DataFrame (statistical_list,columns=['年度','類別','科目','題號','數值'])
    return new_df
    


## 使用自製函數將通過率、鑑別度pdf轉換成dataframe

In [197]:
#非通過率/鑑別度的資料部份去掉
non_data_list = [0,1,39,66,67,68,69,70,71,72,73,74]

#國文48,英語聽力21,英語閱讀41,數學26,社會63,自然54
#來把各科目的各題放回正確位置
subject_questions_num_list = [48,21,41,26,63,54]

year = 110
statistical_type_list = ['鑑別度','通過率']


df_list = []
for i in range(2):
    filename = filename_list[i]
    filename ='會考題目/' + filename
    print(filename)
    statistical_type = statistical_type_list[i]
    
    df_temp = convertPdfToDataframe(non_data_list,subject_questions_num_list,filename)
    df_temp_2 = convertDftoCleandataDf(df_temp,year,statistical_type)
    df_list.append(df_temp_2)
    #print(df)
df_new = pd.concat(df_list)
print(df_new)
                        

會考題目/110各科鑑別度.pdf
2
會考題目/110各科通過率.pdf
2
      年度   類別  科目  題號    數值
0    110  鑑別度  國文   1  0.41
1    110  鑑別度  國文   2  0.44
2    110  鑑別度  國文   3  0.46
3    110  鑑別度  國文   4  0.41
4    110  鑑別度  國文   5  0.47
..   ...  ...  ..  ..   ...
248  110  通過率  自然  50  0.68
249  110  通過率  自然  51  0.79
250  110  通過率  自然  52  0.64
251  110  通過率  自然  53  0.81
252  110  通過率  自然  54  0.78

[506 rows x 5 columns]


In [198]:
#讀取前幾年作好的資料
df = pd.read_csv('整理後的通過率與鑑別度.csv')  
print(df)  

       年度   類別  科目  題號    數值
0     109  通過率  國文   1  0.92
1     109  通過率  國文   2  0.91
2     109  通過率  國文   3  0.95
3     109  通過率  國文   4  0.91
4     109  通過率  國文   5  0.90
...   ...  ...  ..  ..   ...
3487  103  鑑別度  自然  50  0.43
3488  103  鑑別度  自然  51  0.52
3489  103  鑑別度  自然  52  0.49
3490  103  鑑別度  自然  53  0.48
3491  103  鑑別度  自然  54  0.44

[3492 rows x 5 columns]


In [201]:
#合併前幾年和今年的資料
df_combine = pd.concat([df, df_new])

In [203]:
df_combine['科目'].unique()

array(['國文', '英語', '數學', '社會', '自然', '英語聽力', '英語(聽力)', '英語(閱讀)'],
      dtype=object)

In [204]:
df_combine.loc[df_combine['科目'] == '英語(聽力)', '科目'] = '英語聽力'
df_combine.loc[df_combine['科目'] == '英語(閱讀)', '科目'] = '英語'

In [205]:
df_combine['科目'].unique()

array(['國文', '英語', '數學', '社會', '自然', '英語聽力'], dtype=object)

In [206]:
df_combine.to_csv('整理後的通過率與鑑別度.csv', index=False,encoding="utf_8_sig")

# 處理題目，最後要存到 從pdf直存df.csv

In [211]:
import re

In [223]:
for i in range(2,7):
    filename = filename_list[i]
    print(filename)
    subject = re.findall(r'(?=_(.*).pdf)'  ,filename,re.DOTALL)
    subject = subject[0]
    
    print(subject)


110P_Chinese.pdf
Chinese
110P_English.pdf
English
110P_Math.pdf
Math
110P_Society.pdf
Society
110P_Nature.pdf
Nature


In [224]:
#解析題目

pdf_file ='會考題目/' + filename     #pdf檔路徑及檔名
print(filename)
text = extract_content(pdf_file)
text

question_list = []

for i in range(2,7):
    filename = filename_list[i]
    pdf_file ='會考題目/' + filename 
    
    subject = re.findall(r'(?=_(.*).pdf)'  ,filename,re.DOTALL)
    subject = subject[0]            
    text = extract_content(pdf_file)


    #matches = re.findall('\\n(\d{1,2}\.{1}\s{2}.*?？)', text, re.DOTALL)
    #matches = re.findall(r'(?=\n(\d{1,2}\.{1}\s{2}.*?)\n\d{1,2}\.{1}\s{2})', text, re.DOTALL)
    matches = re.findall(r'(?=\n(\d{1,2}\.{1}\s{2}.*?\(D\).*?\n))', text, re.DOTALL)

    print(matches)

    #取出題號
    question_num_list = []
    for question in matches:
        question_num = re.match('^(\d{1,2})', question, re.DOTALL)
        question_num_list.append(question_num[0])

    #將各題分開放在list            
    for i in range(len(matches)):
        if subject == 'English':
            match = matches[i].replace('\n', '').replace('\r', '').replace('　', '')
        else:
            match = matches[i].replace('\n', '').replace('\r', '').replace(' ', '').replace('　', '')
        question = [year, subject, question_num_list[i] , match]
        print(year, subject, question_num_list[i] , match)
        question_list.append(question)

questions_df = pd.DataFrame(question_list , columns = ['年度', '科目','題號','題目'])
print(questions_df)


110P_Nature.pdf
15


15
['1.      \n下列文句中的連詞，何者使用最恰當？\n(A)\n即使生活變得富裕，也不應該浪費\n(B)\n除非是很小的錯誤，也可能釀成大害\n(C)\n只有從根本解決，就能為人民改善生活\n(D)\n', '2.    \n「人生是短暫的，死亡是自然的，認清了這點，活著的時候才可以真正的投入。」\n這句話的涵義，與下列何者最接近？\n(A)\n經歷死亡幽谷，才能發現活著的可貴\n(B)\n能將死亡置之度外的人往往雖死猶生\n(C)\n生命的價值在於不斷挑戰自我，安於現狀等於死亡\n(D)\n', '3.     \n這是某相機使用說明書的一部分：\n對於以下主體，可能無法成功自動對焦：\n\uf077 與背景色彩相近的主體\n\uf077 自動對焦點同時覆蓋遠近主體\n\uf077 重複的圖案\n    \n\u3000 根據以上說明，這臺相機在相同的設定下，下列哪一個景物最容易自動對焦？\n(A) (B) (C) (D)\n', '4.  \n下列選項「」中的字，何組讀音相同？\n(A)\n「恪」守本分／獨創一「格」\n(B)\n「抵」死不從／蒞臨府「邸」\n(C)\n字「斟」句酌／「勘」正訛誤\n(D)\n', '6.      \n(cid:2294)(cid:2620)\n(cid:1258)(cid:3817)\n  \n(cid:2889)(cid:2256)(cid:2294)(cid:2620)(cid:514)(cid:1091)(cid:977)(cid:6025)(cid:4660)(cid:2256)(cid:4439)(cid:3919) (cid:2735)(cid:1270)(cid:3294)(cid:2036)(cid:6103)(cid:1952)(cid:3654)(cid:4899)(cid:514)(cid:1091)(cid:12934)(cid:4425)\n(cid:1270)(cid:1056)(cid:3884)(cid:1317)(cid:2410)(cid:1952)(cid:2294)(cid:4168)(cid:514)(cid:1091)(cid:5120)(cid:1317) (cid:4741)(cid:5762)(cid:3603)(cid:5919)(cid:1952)(cid:1

13
['1.  In the picture, the boy is            the old man.  \n(A) smiling at\n(B) dancing with\n(C) cheering for\n(D) bowing to \n', '2.  Listen!  The baby            in the bedroom.  Why don’t you go in and take a look?\n(A) cried  (B) cries   (C) is crying  (D) will cry\n', '3.  Jill is            that the city park is closed for the music festival because now she can’t \njog there.\n(A) excited  (B) proud  (C) scared  (D) unhappy\n', '4.  Steven wants to be a      , because he loves to watch people enjoy the food he prepares.\n(A) cook  (B) doctor  (C) driver       (D) farmer\n', '5.  Paul misses his parents a lot.  He            them since he came to work in Taiwan a \nyear ago.\n(A) didn’t see  (B) doesn’t see  (C) hasn’t seen  (D) won’t see \n', '6.  Our teacher Ms. Wu seldom laughs, but when she           , everyone in the same \nbuilding can hear her.    \n(A) can   (B) does  (C) has  (D) will\n', '7.  My sister is coming to my home today.  She            with me for a week.\n

['1.  圖( 一 ) 的坐標平面上有A、B、C、D四點。根據 A\n( )\n\u3000\u3000圖 一 中各點位置判斷，哪一個點在第二象限？\nx\nO\n(A) A B\n(B) B\nD\n(C) C\nC\n(D) D\n', '2.   (−8) + (−2) × (−3)\n算式 之值為何？\n(A) −14\n(B) −2\n(C) 18\n(D) 30\n', '3.     x = a  y = b  a + b \n若二元一次聯立方程式 \u3000\u3000\u3000\u3000\u3000\u3000的解為 ， ，則 之值為何？\n 6y − x = 10\n \n(A) −15\n(B) −3\n(C) 5\n(D) 25\n', '4.  ( )     ABCD   ∆BDE     A  BE     ABCD\n如圖 二 ， 矩形 、 中 ， 點在 上。若矩形 的面積\n 20   ∆ BDE  24     ∆ ADE\n為 ， 的面積為 ，則 的面積為何？\nE\n(A) 10\n(B) 12\n(C) 14 A D\n(D) 16\n', '5.  56 是 53 的多少倍？\n(A) 2\n(B) 3\n(C) 25\n(D) 125\n', '6.  下列等式何者不成立？\n(A) 4 3 + 2 3 = 6 3\n(B) 4 3 − 2 3 = 2 3\n(C) 4 3 × 2 3 = 8 3\n(D) 4 3 ÷ 2 3 = 2\n', '7.  已知纜車從起點行駛到終點需花費 8 分鐘 ， 圖 ( 三 ) 表示行駛過程中纜車的\n海拔高度與行駛時間的關係。\n(cid:2631) 350\n(cid:1816) 300\n250\n(cid:2863)\n200\n(cid:2123)\n150\n((cid:998)\n  100\n \n (cid:1025)\n   50\n \n \n) 0\n \n0 1 2 3 4 5 6 7 8\n(cid:1353)(cid:5037)(cid:2580)(cid:3796)((cid:1001)    )\n( )\n圖 三\n( )    \n根據圖 三 判斷 ， 下列敘述何者正確？\n(A)  300  4\n終點的海拔高度比起點高 公尺，行駛時間的

['1.  ( ) (cid:6332)(cid:1110)(cid:2321)(cid:2541)\n觀光局推出探索圖一中十個島嶼的活動。 (cid:27027)(cid:18749)\n小蘭的外國朋友想來臺灣進行島嶼旅遊， (cid:6331)(cid:1870)(cid:3269)(cid:2541)\n她為友人規畫了下列行程：先浮潛、欣賞\n當地原住民的地下屋及品嘗飛魚料理；接 (cid:25016)(cid:25959) (cid:6323)(cid:5370)(cid:973)(cid:2541)\n(cid:6330)(cid:2665)(cid:5392)\n著欣賞柱狀玄武岩及石滬景觀；最後欣賞\n(cid:6329)(cid:1250)(cid:1621)(cid:5392)\n國家公園內的戰地風光及風獅爺文化。上\n(cid:6328)(cid:1370)(cid:5392)\n述行程依序應為右圖中哪些島嶼？\n(cid:6327)(cid:925)(cid:2332)(cid:5392)\n(cid:16245)(cid:15933)\n(A)\n(cid:6324)(cid:4519)(cid:2541)\n(B)\n(cid:6326)(cid:2676)(cid:3177)(cid:5392)\n(cid:6325)(cid:6093)(cid:5392)\n(C)\n( )\n圖 一\n(D)\n', '2.  ( )\n表一為臺灣四家企業為了提升競爭 企業 採行的方法\n力而採行的方法，其中何者主要是為\n甲 製鞋廠從中國沿海遷廠到內陸\n了達到增加產品附加價值的目的？\n乙 紡織廠研發吸溼排汗的機能布料\n(A)\n甲\n丙 煉鋼廠設置在高雄臨海工業區內\n(B)\n乙\n丁 電子廠自東南亞引進大量外籍勞工\n(C)\n丙\n(D)\n', '3.  ( ) 1946 1995\n圖 二 為 年至 年臺灣月平均晒鹽產量統計圖，\n((cid:4068)5\n其中某些月分的產量明顯較低。根據臺灣晒鹽產區的 (cid:998)4\n(cid:5074)3\n氣候特性判斷，造成這些月分產量較低的氣候因素最\n)2\n可能為下列何者？ 1\n(A)   (B) 0\n蒸發較弱 降雨量較多 1 3 5 7 9 11((cid:1044))\n

['1.  ( )\n表 一 為阿梅整理的甲、乙二項使用上皿天平時的注意事項及其對應原因，關於\n其對應原因是否合理，下列敘述何者正確？\n( )\n表 一\n \n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\n(cid:1881)(cid:3904)(cid:1635)(cid:3816) (cid:4345)(cid:5384)(cid:2458)(cid:1250)\n(cid:1176) (cid:3571)(cid:3782)(cid:1922)(cid:2068)(cid:2043)(cid:500)(cid:5384)(cid:2874)(cid:1267)(cid:5660)(cid:4255)(cid:2895)(cid:1378)(cid:1222)(cid:3571)(cid:3782) (cid:1105)(cid:3559)(cid:1023)(cid:3571)(cid:3782)(cid:4966)(cid:3782)(cid:2580)(cid:1937)(cid:4580)(cid:2529)\n(cid:908) (cid:2558)(cid:1687)(cid:2694)(cid:4856)(cid:2580)(cid:972)(cid:1105)(cid:1172)(cid:1035)(cid:2558)(cid:1687)(cid:500)(cid:5384)(cid:1172)(cid:2694)(cid:4856)(cid:1448)(cid:2558)(cid:1687) (cid:1105)(cid:3559)(cid:1023)(cid:2694)(cid:4856)(cid:1171)(cid:6028)(cid:1937)(cid:5138)(cid:3950)\n(A)     (B)\n兩者皆合理 兩者皆不合理\n(C)   (D)\n', '2. \n小茹統計某漁港每日的潮汐水位高度資料，她發現此漁港最高的滿潮水位高\n2 2\n於平均海平面高度 公尺，而最低的乾潮水位低於平均海平面高度 公尺。根據\n小茹的統計資料，此漁港的潮差高度不可能為下列何者？\n(A) 2     (B)

In [225]:
questions_df.loc[questions_df['科目'] == 'Chinese', '科目'] = '國文'
questions_df.loc[questions_df['科目'] == 'English', '科目'] = '英語'
questions_df.loc[questions_df['科目'] == 'Math', '科目'] = '數學'
questions_df.loc[questions_df['科目'] == 'Society', '科目'] = '社會'
questions_df.loc[questions_df['科目'] == 'Nature', '科目'] = '自然'
print(questions_df)  

      年度  科目  題號                                                 題目
0    110  國文   1  1.下列文句中的連詞，何者使用最恰當？(A)即使生活變得富裕，也不應該浪費(B)除非是很小的...
1    110  國文   2  2.「人生是短暫的，死亡是自然的，認清了這點，活著的時候才可以真正的投入。」這句話的涵義，與...
2    110  國文   3  3.這是某相機使用說明書的一部分：對於以下主體，可能無法成功自動對焦：與背景色彩相近的主體...
3    110  國文   4  4.下列選項「」中的字，何組讀音相同？(A)「恪」守本分／獨創一「格」(B)「抵」死不從／蒞...
4    110  國文   6  6.(cid:2294)(cid:2620)(cid:1258)(cid:3817)(cid...
..   ...  ..  ..                                                ...
222  110  自然  50  50.()根據表十一，下列何者最可能是小敏想探討的關係？(A)大氣壓力與茶包長度的關係(B)...
223  110  自然  51  51.根據本文，若小敏在向同學說明「茶包天燈」成因後，想要再舉一個科學原理相同的例子，則下列...
224  110  自然  52  52.根據本文，下列有關甲類生物的推論，何者最合理？(A)與三角龍同時期滅亡(B)與劍龍有共...
225  110  自然  53  53.根據上述資料，若在某地最上方的岩層中可找到甲類生物化石，且此地岩層未曾倒轉，下列何者最...
226  110  自然  54  54.()根據表十二，下列敘述中，何者是有可能發生的？(A)分析暴龍的進食殘骸，發現暴龍也會...

[227 rows x 4 columns]


In [226]:
#開啟舊的題庫
question_df_old = pd.read_csv('從pdf直存df.csv')
question_df_old



,年度,科目,題號,題目
0,104,國文,1,1.小敬搬新家，阿嬌欲送禮金表達祝賀之意。下列題辭，何者最適合寫在賀儀封套上？(A)近悅遠來...
1,104,國文,2,2.下列文句，何者文意最為通暢？(A)因為你不去，我還是要去(B)他年紀既然小，膽子卻不小(...
2,104,國文,3,3.「當部分亞洲國家為了生產足夠的稻米而焦頭爛額之際，泰國稻米的產量卻多到一年出口九百萬噸，...
3,104,國文,4,4.下列詞語「」中的字音，何組相同？(A)風聲鶴「唳」／暴「戾」之氣(B)「揖」讓而升／查「...
4,104,國文,5,5.「時然後言，人不厭其言。」這句話的涵義，與下列何者最接近？(A)沉默是金，可免禍從口出(...
...,...,...,...,...
1361,109,自然,50,50.根據本文，肺炎鏈球菌不具有下列何者？(A)DNA(B)細胞質(C)細胞膜(D)細胞核
1362,109,自然,51,51.已知紅血球為血液中數量最多的血球，根據本文，關於甲、乙、丙的推論，下列何者正確？(A)...
1363,109,自然,52,52.已知化合物X是含有−OH原子團的有機化合物，上述畫線處的反應說明，下列敘述何者正確？(...
1364,109,自然,53,53.上述農民「使用其他方式」，最可能是下列何者？(A)在竹筍生長處覆蓋透明塑膠布，每日陽光...


In [228]:
#合併新的和舊的題庫
question_df_combine = pd.concat([question_df_old, questions_df])
question_df_combine


,年度,科目,題號,題目
0,104,國文,1,1.小敬搬新家，阿嬌欲送禮金表達祝賀之意。下列題辭，何者最適合寫在賀儀封套上？(A)近悅遠來...
1,104,國文,2,2.下列文句，何者文意最為通暢？(A)因為你不去，我還是要去(B)他年紀既然小，膽子卻不小(...
2,104,國文,3,3.「當部分亞洲國家為了生產足夠的稻米而焦頭爛額之際，泰國稻米的產量卻多到一年出口九百萬噸，...
3,104,國文,4,4.下列詞語「」中的字音，何組相同？(A)風聲鶴「唳」／暴「戾」之氣(B)「揖」讓而升／查「...
4,104,國文,5,5.「時然後言，人不厭其言。」這句話的涵義，與下列何者最接近？(A)沉默是金，可免禍從口出(...
...,...,...,...,...
222,110,自然,50,50.()根據表十一，下列何者最可能是小敏想探討的關係？(A)大氣壓力與茶包長度的關係(B)...
223,110,自然,51,51.根據本文，若小敏在向同學說明「茶包天燈」成因後，想要再舉一個科學原理相同的例子，則下列...
224,110,自然,52,52.根據本文，下列有關甲類生物的推論，何者最合理？(A)與三角龍同時期滅亡(B)與劍龍有共...
225,110,自然,53,53.根據上述資料，若在某地最上方的岩層中可找到甲類生物化石，且此地岩層未曾倒轉，下列何者最...


In [229]:
#回存入'從pdf直存df.csv'
question_df_combine.to_csv('從pdf直存df.csv', index=False,encoding="utf_8_sig")

In [207]:
#接下來執行B03分析通過率和鑑別度題目

# 以下是測試

In [110]:
#測試
#取得pdf內文為 text
filename = filename_list[0]
pdf_file ='會考題目/' + filename     #pdf檔路徑及檔名
print(filename)
text = extract_content(pdf_file)
text

110各科鑑別度.pdf
2


'110年國中教育會考各題鑑別度\n題序 國文 英語(聽力) 英語(閱讀) 數學 社會 自然\n1 0.41 0.36 0.61 0.46 0.38 0.33\n2 0.44 0.54 0.50 0.52 0.45 0.40\n3 0.46 0.57 0.65 0.60 0.41 0.58\n4 0.41 0.56 0.50 0.49 0.50 0.48\n5 0.47 0.64 0.61 0.52 0.41 0.54\n6 0.41 0.67 0.63 0.60 0.33 0.53\n7 0.48 0.63 0.62 0.44 0.55 0.55\n8 0.58 0.70 0.61 0.55 0.54 0.53\n9 0.51 0.65 0.61 0.53 0.57 0.52\n10 0.52 0.64 0.59 0.70 0.53 0.43\n11 0.52 0.63 0.56 0.58 0.44 0.51\n12 0.57 0.63 0.46 0.49 0.41 0.48\n13 0.49 0.49 0.47 0.57 0.50 0.65\n14 0.55 0.68 0.49 0.55 0.47 0.50\n15 0.46 0.64 0.61 0.34 0.31 0.64\n16 0.43 0.63 0.61 0.57 0.44 0.36\n17 0.53 0.64 0.61 0.54 0.53 0.51\n18 0.54 0.60 0.64 0.58 0.53 0.52\n19 0.51 0.49 0.71 0.57 0.53 0.61\n20 0.50 0.51 0.63 0.36 0.46 0.58\n21 0.35 0.46 0.63 0.57 0.62 0.60\n22 0.49 0.65 0.55 0.49 0.44\n23 0.41 0.66 0.50 0.54 0.41\n24 0.55 0.63 0.53 0.44 0.59\n25 0.43 0.58 0.31 0.50 0.37\n26 0.52 0.67 0.43 0.39 0.62\n27 0.28 0.40 0.50 0.60\n28 0.52 0.59 0.39 0.52\n29 0.45 0.66 0.50 0.48\n30 0.49 0.54 0.52 0.62\n31 0.4

In [112]:
for i in range(len(new_text)):
    print(i,'----',new_text[i])

0 ---- ['110年國中教育會考各題鑑別度']
1 ---- ['題序', '國文', '英語(聽力)', '英語(閱讀)', '數學', '社會', '自然']
2 ---- ['1', '0.41', '0.36', '0.61', '0.46', '0.38', '0.33']
3 ---- ['2', '0.44', '0.54', '0.50', '0.52', '0.45', '0.40']
4 ---- ['3', '0.46', '0.57', '0.65', '0.60', '0.41', '0.58']
5 ---- ['4', '0.41', '0.56', '0.50', '0.49', '0.50', '0.48']
6 ---- ['5', '0.47', '0.64', '0.61', '0.52', '0.41', '0.54']
7 ---- ['6', '0.41', '0.67', '0.63', '0.60', '0.33', '0.53']
8 ---- ['7', '0.48', '0.63', '0.62', '0.44', '0.55', '0.55']
9 ---- ['8', '0.58', '0.70', '0.61', '0.55', '0.54', '0.53']
10 ---- ['9', '0.51', '0.65', '0.61', '0.53', '0.57', '0.52']
11 ---- ['10', '0.52', '0.64', '0.59', '0.70', '0.53', '0.43']
12 ---- ['11', '0.52', '0.63', '0.56', '0.58', '0.44', '0.51']
13 ---- ['12', '0.57', '0.63', '0.46', '0.49', '0.41', '0.48']
14 ---- ['13', '0.49', '0.49', '0.47', '0.57', '0.50', '0.65']
15 ---- ['14', '0.55', '0.68', '0.49', '0.55', '0.47', '0.50']
16 ---- ['15', '0.46', '0.64', '0.61', '0.34', '0.

In [113]:
columns_name = new_text[1]
#將非資料的部份去掉
non_data_list = [0,1,39,66,67,68,69,70,71,72,73,74]

temp_text = []
for i in range(len(new_text)):
    if i not in non_data_list:
        temp_text.append(new_text[i])
print(temp_text)
new_text = temp_text.copy()


[['1', '0.41', '0.36', '0.61', '0.46', '0.38', '0.33'], ['2', '0.44', '0.54', '0.50', '0.52', '0.45', '0.40'], ['3', '0.46', '0.57', '0.65', '0.60', '0.41', '0.58'], ['4', '0.41', '0.56', '0.50', '0.49', '0.50', '0.48'], ['5', '0.47', '0.64', '0.61', '0.52', '0.41', '0.54'], ['6', '0.41', '0.67', '0.63', '0.60', '0.33', '0.53'], ['7', '0.48', '0.63', '0.62', '0.44', '0.55', '0.55'], ['8', '0.58', '0.70', '0.61', '0.55', '0.54', '0.53'], ['9', '0.51', '0.65', '0.61', '0.53', '0.57', '0.52'], ['10', '0.52', '0.64', '0.59', '0.70', '0.53', '0.43'], ['11', '0.52', '0.63', '0.56', '0.58', '0.44', '0.51'], ['12', '0.57', '0.63', '0.46', '0.49', '0.41', '0.48'], ['13', '0.49', '0.49', '0.47', '0.57', '0.50', '0.65'], ['14', '0.55', '0.68', '0.49', '0.55', '0.47', '0.50'], ['15', '0.46', '0.64', '0.61', '0.34', '0.31', '0.64'], ['16', '0.43', '0.63', '0.61', '0.57', '0.44', '0.36'], ['17', '0.53', '0.64', '0.61', '0.54', '0.53', '0.51'], ['18', '0.54', '0.60', '0.64', '0.58', '0.53', '0.52'], 

In [114]:
#國文48,英語聽力21,英語閱讀41,數學26,社會63,自然54
#把各科目的各題放回正確位置
subject_questions_num_list = [48,21,41,26,63,54]

temp_text = new_text.copy()
#個別處理各科
for i in range(len(subject_questions_num_list)):
    for j in range(len(temp_text)):
        if j+1 > subject_questions_num_list[i]:
            temp_text[j].insert(i+1,'')

print(temp_text)
    


[['1', '0.41', '0.36', '0.61', '0.46', '0.38', '0.33'], ['2', '0.44', '0.54', '0.50', '0.52', '0.45', '0.40'], ['3', '0.46', '0.57', '0.65', '0.60', '0.41', '0.58'], ['4', '0.41', '0.56', '0.50', '0.49', '0.50', '0.48'], ['5', '0.47', '0.64', '0.61', '0.52', '0.41', '0.54'], ['6', '0.41', '0.67', '0.63', '0.60', '0.33', '0.53'], ['7', '0.48', '0.63', '0.62', '0.44', '0.55', '0.55'], ['8', '0.58', '0.70', '0.61', '0.55', '0.54', '0.53'], ['9', '0.51', '0.65', '0.61', '0.53', '0.57', '0.52'], ['10', '0.52', '0.64', '0.59', '0.70', '0.53', '0.43'], ['11', '0.52', '0.63', '0.56', '0.58', '0.44', '0.51'], ['12', '0.57', '0.63', '0.46', '0.49', '0.41', '0.48'], ['13', '0.49', '0.49', '0.47', '0.57', '0.50', '0.65'], ['14', '0.55', '0.68', '0.49', '0.55', '0.47', '0.50'], ['15', '0.46', '0.64', '0.61', '0.34', '0.31', '0.64'], ['16', '0.43', '0.63', '0.61', '0.57', '0.44', '0.36'], ['17', '0.53', '0.64', '0.61', '0.54', '0.53', '0.51'], ['18', '0.54', '0.60', '0.64', '0.58', '0.53', '0.52'], 

In [115]:
for i in range(len(temp_text)):
    print(len(temp_text[i]),'-----',temp_text[i])

7 ----- ['1', '0.41', '0.36', '0.61', '0.46', '0.38', '0.33']
7 ----- ['2', '0.44', '0.54', '0.50', '0.52', '0.45', '0.40']
7 ----- ['3', '0.46', '0.57', '0.65', '0.60', '0.41', '0.58']
7 ----- ['4', '0.41', '0.56', '0.50', '0.49', '0.50', '0.48']
7 ----- ['5', '0.47', '0.64', '0.61', '0.52', '0.41', '0.54']
7 ----- ['6', '0.41', '0.67', '0.63', '0.60', '0.33', '0.53']
7 ----- ['7', '0.48', '0.63', '0.62', '0.44', '0.55', '0.55']
7 ----- ['8', '0.58', '0.70', '0.61', '0.55', '0.54', '0.53']
7 ----- ['9', '0.51', '0.65', '0.61', '0.53', '0.57', '0.52']
7 ----- ['10', '0.52', '0.64', '0.59', '0.70', '0.53', '0.43']
7 ----- ['11', '0.52', '0.63', '0.56', '0.58', '0.44', '0.51']
7 ----- ['12', '0.57', '0.63', '0.46', '0.49', '0.41', '0.48']
7 ----- ['13', '0.49', '0.49', '0.47', '0.57', '0.50', '0.65']
7 ----- ['14', '0.55', '0.68', '0.49', '0.55', '0.47', '0.50']
7 ----- ['15', '0.46', '0.64', '0.61', '0.34', '0.31', '0.64']
7 ----- ['16', '0.43', '0.63', '0.61', '0.57', '0.44', '0.36']
7

In [116]:
#將new_text轉為 df
from pandas import DataFrame

df = DataFrame (temp_text,columns=columns_name)
df

,題序,國文,英語(聽力),英語(閱讀),數學,社會,自然
0,1,0.41,0.36,0.61,0.46,0.38,0.33
1,2,0.44,0.54,0.50,0.52,0.45,0.40
2,3,0.46,0.57,0.65,0.60,0.41,0.58
3,4,0.41,0.56,0.50,0.49,0.50,0.48
4,5,0.47,0.64,0.61,0.52,0.41,0.54
...,...,...,...,...,...,...,...
58,59,,,,,0.38,
59,60,,,,,0.46,
60,61,,,,,0.53,
61,62,,,,,0.45,


In [146]:
#把df整型成 年度	類別	科目	題號	數值
static_list = []
for index, row in df.iterrows():
    #print(row['c1'], row['c2'])
    print(row['國文'])

0.41
0.44
0.46
0.41
0.47
0.41
0.48
0.58
0.51
0.52
0.52
0.57
0.49
0.55
0.46
0.43
0.53
0.54
0.51
0.50
0.35
0.49
0.41
0.55
0.43
0.52
0.28
0.52
0.45
0.49
0.43
0.57
0.47
0.45
0.44
0.45
0.48
0.53
0.51
0.47
0.39
0.54
0.49
0.40
0.59
0.47
0.39
0.42

















In [151]:
for column in df:
    #print(df[column])
    print(column[])


題序
國文
英語(聽力)
英語(閱讀)
數學
社會
自然


In [167]:
#年度	類別	科目	題號	數值
year = 110
statistical_type = '鑑別度'
statistical_list = []
for idx, column in enumerate(df):
    #print(idx, column)
    if idx > 0:
        for idx_2, row in enumerate(df[column]):
            if len(row) > 0:
                #print(year,statistical_type,column,idx_2+1,float(row))
                statistical_list.append([ year,statistical_type,column,idx_2+1,float(row) ])

#print(statistical_list)
new_df = DataFrame (statistical_list,columns=['年度','類別','科目','題號','數值'])
print(new_df)
            
    

      年度   類別  科目  題號    數值
0    110  鑑別度  國文   1  0.41
1    110  鑑別度  國文   2  0.44
2    110  鑑別度  國文   3  0.46
3    110  鑑別度  國文   4  0.41
4    110  鑑別度  國文   5  0.47
..   ...  ...  ..  ..   ...
248  110  鑑別度  自然  50  0.61
249  110  鑑別度  自然  51  0.55
250  110  鑑別度  自然  52  0.57
251  110  鑑別度  自然  53  0.53
252  110  鑑別度  自然  54  0.49

[253 rows x 5 columns]
